In [ ]:
! pip install trueskill

In [ ]:
import trueskill
from trueskill import Rating, rate
import pandas as pd
#import xlwings as xw
#import xlsxwriter
import os
import pickle
import math
import numpy as np

In [ ]:
trueskill.setup(backend='mpmath')

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=10.0%, backend='mpmath')

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')   
os.chdir("/content/gdrive/My Drive/")
BASE_DIR =  'MIE490: Capstone/Data/chronological/'

Mounted at /content/gdrive


In [ ]:
'''
Define the Athlete class to represent each athlete
'''
class Athlete:
    def __init__(self, fisCode, name, gender, country, rating, sigma, placement, birthdate, start_year, end_year, initial_level):
        self.fisCode = fisCode
        self.name = name
        self.gender = gender
        self.country = country
        self.rating = rating
        self.sigma = sigma
        self.placement = placement
        self.birthdate = birthdate
        self.start_year = start_year
        self.end_year = start_year
        self.initial_level = initial_level
        self.records = [None]*30


    def getInfo(self):
        return [self.fisCode, self.name, self.gender, self.birthdate, self.start_year, self.end_year, self.initial_level, self.country, self.rating]

    def updatePlacement(self, newPlacement):
        self.placement = newPlacement
    
    def getRating(self):
        return self.rating
    
    def getPlacement(self):
        return self.placement

    def updateRating(self,year):
        self.records[year-self.start_year] = self.rating
        self.end_year = year

    def __eq__(self, other):
        return self.fisCode == other.fisCode 

    def __hash__(self):
        return hash((self.name, self.fisCode))

    def __str__(self):
        return "{}: fisCode: {}, rating: {}, gender: {}, country: {}".format(self.name, self.fisCode, self.rating, self.gender, self.country)

In [ ]:
'''
A class of competition representation. Process all participated athletes in a list
'''
class Competition:
    def __init__(self):
        self.competitors = []

    def addCompetitor(self, athlete):
        self.competitors.append(athlete)

In [ ]:
'''
Process one competition data into a competition class object. 
Each athlete is updated with his/her rank in this competition.
After processing all athletes, their competition ranks  will be used to calculate Elo score. 
'''

def competitionFileParser(df, comp_year):
    # df is the dataframe that contains competition data for one competition
    # date = df.loc[0,'Race Date1']

    gender = df['Gender (Races!Sb)'].values[0]
    comp_level = df['Cup Type'].values[0]

    competition = Competition()
    for row in (df.index):
        fisCode = df.loc[row,'Fis Code']
        placement = df.loc[row,'Rank']
        birthdate = df.loc[row, 'Birthdate']
        # Initialize a new Athlete object
        if fisCode not in athlete_dict.keys():
            # NOTE: some athlete has no name...
            name = df.loc[row,'Athlete Name']
            country = df.loc[row,'Nation']
            # placement = df.loc[row,'Rank']
            if comp_level == 'WSC' or comp_level == 'WC':
                # Championship
                rating = 500
                sigma = rating/3
            elif comp_level == 'NAC' or comp_level == 'WJC':
                rating = 300
                sigma = rating/3
            elif comp_level == 'NC' or comp_level == 'NC/FI':
                rating = 150
                sigma = rating/3
            else:
                rating = 100
                sigma = rating/3

            athlete = Athlete(fisCode, name, gender, country, rating, sigma, placement,birthdate, comp_year, comp_year, comp_level)
            athlete_dict[fisCode] = athlete
        # update athlete's rank in current competition
        else:
            athlete = athlete_dict.get(fisCode)
            athlete.updatePlacement(placement)

        # add athlete to the current competition class
        competition.addCompetitor(athlete)
        
    return competition

In [ ]:
def calculateRating(competition,comp_year):

    ratingList = []
    rankList = []
    N = len(competition)
    for i in range(0, N):
        player = competition[i]
        ratingList.append([Rating(mu=player.rating,sigma=player.sigma)])
        rankList.append(player.getPlacement()-1)

    newRatings = rate(ratingList, ranks=rankList)    

    for i in range(0, N-1):
        player = competition[i]
        player.rating = newRatings[i][0].mu
        player.sigma = newRatings[i][0].sigma
        player.updateRating(comp_year)

In [ ]:
'''
Use the training data until the end_Date to calculate the Elo scores for all the 
athletes.

folderName: The path to either Male or Female competition data
'''

def train_system(folderName, end_date):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    for file in files:
        # discard qualification competition
        # if file == 'M_HP_QL.xlsx':
        #      continue
        end_year = int(str(end_date)[:4])
        file_year = int(file.split('_')[-1][:4])
        if file_year > end_year:
            continue 
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            ## sheet_names.sort(reverse = False)
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # ground truth ranking data starts from 20131205
                # use all data prior 20131205 as training data
                if comp_date < end_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athlete who DNS, DNF, DSQ
                  df = df[df['Status']== 'QLF']
                  
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run Rating calculation for this competition,
                  # and update athletes' Rating score, store their track of Rating scores
                  calculateRating(competition.competitors, comp_year)

In [ ]:
'''
Sort the athlete dictionary and return a DataFrame with sorted athlete information
'''
def sort_athlete(athlete_dict):
    sorted_athlete = sorted(athlete_dict.items(), key=lambda x: (x[1].rating-3*x[1].sigma), reverse=True)

    temp = []
    for fiscode, athlete in sorted_athlete:
      temp.append(athlete.getInfo())

    return pd.DataFrame(temp, columns = ['fiscode', 'name', 'gender', 'birthdate','First competition year','Last competition year', 'First competition level', 'country', 'rating'])

In [ ]:
official_ranking_folder = 'MIE490: Capstone/Data/Ranking'
official_ranking_file = 'HP_M.xlsx'
official_ranking_file_W = 'HP_W.xlsx'

xls_ranking_M = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file)
sheet_names_M = xls_ranking_M.sheet_names

xls_ranking_W = pd.ExcelFile(official_ranking_folder +'/' + official_ranking_file_W)
sheet_names_W = xls_ranking_W.sheet_names

In [ ]:
'''
Search for the ranking data that is the most recent to the given comp_date.
The ranking data comes from the given official athlete ranking list.

Return the ranking data that is closest to the competition date.

This ranking data will be used as benchmark for comparsion to our point system

sheet_names: a list of sheet names represents the ranking data. In order of most recent ranking date, that is most recent to oldest
'''
def search_official_ranking(xls, sheet_names, comp_date):
    for sheet in sheet_names:
        sheet_date = sheet.split('_')[-1]
        # found the ranking data prior to the competition date
        if comp_date > sheet_date:
            return xls.parse(sheet,header = 0)

In [ ]:
'''
Evaluate the system starting from the start_date to validation_end_date.
After evaluate athletes basen on their current competition results and prior Elo scores,
the ranking system update elo scores for all the athletes to fully reflect their true skill based on thier performance.

start_date should be at least > 20131205 because the official published ranking data is only availble after this time

xls_ranking and ranking_sheet_names help the system to find the most recent published offcial ranking
The offcial ranking is used to evaluate the accuracy of the ranking system.

''' 
def evaluate_system(folderName, start_date, validation_end_date, xls_ranking, ranking_sheet_names, show_yearly = False):
    files = os.listdir(folderName)
    # sort files in reverse chronological order 
    files.sort(reverse = False)
    # Go through each competition data
    validation_result = pd.DataFrame(columns=['competition', 'Rating accuracy', 'Official accuracy','Absolute Rating Dev', 'Absolute Official Dev','Pairwise Rating Dev', 'Pairwise Official Dev'])
    testing_result = pd.DataFrame(columns=['competition', 'Rating accuracy', 'Official accuracy','Absolute Rating Dev', 'Absolute Official Dev', 'Pairwise Rating Dev', 'Pairwise Official Dev'])

    start_year = int(str(start_date)[:4])
    # files are in the chronological order from oldest to most recent
    for file in files:
        file_year = int(file.split('_')[-1][:4])
        if file_year < start_year:
            continue                                                                                                                                         
        print('Currently processing file: {}\n'.format(file))
        yearly_result = pd.DataFrame(columns=['competition', 'Rating accuracy', 'Official accuracy','Absolute Rating Dev', 'Absolute Official Dev', 'Pairwise Rating Dev', 'Pairwise Official Dev'])
        if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
            xls = pd.ExcelFile(folderName +'/' + file)
            sheet_names = xls.sheet_names
            #sheet_names.sort(reverse = False)
            
            ## each sheet is a competition in that competition level
            for sheet in sheet_names:
                comp_date = int(sheet.split('_')[-1])
                comp_year = int(sheet.split('_')[-1][0:4])
                # Note: ground truth ranking data starts from 20131205
                # Evaluate the system starting from the start_date
                if comp_date >= start_date:
                  df = xls.parse(sheet,header = 0)
                  # remove the athletes who have not finished the competition
                  df = df[df['Status']== 'QLF']
                  # get the official ranking data
                  official_ranking = search_official_ranking(xls_ranking, ranking_sheet_names, sheet.split('_')[-1])
                  # feed the competition data and official ranking to compute the accuracy
                  Rating_accuracy, official_accuracy, Rating_deviation, official_deviation, Rating_pairwise_deviation, official_pairwise_deviation = evaluation(df, official_ranking)
                  #print('Rating: {}, official: {}'.format(Rating_accuracy, official_accuracy))
                  yearly_result = yearly_result.append({'competition':sheet,
                                          'Rating accuracy': Rating_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Rating Dev': Rating_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Rating Dev': Rating_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)
                  if comp_date <= validation_end_date:
                      validation_result = validation_result.append({'competition':sheet,
                                          'Rating accuracy': Rating_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Rating Dev': Rating_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Rating Dev': Rating_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True)
                  else:
                      testing_result = testing_result.append({'competition':sheet,
                                          'Rating accuracy': Rating_accuracy,
                                          'Official accuracy': official_accuracy,
                                          'Absolute Rating Dev': Rating_deviation,
                                          'Absolute Official Dev': official_deviation,
                                          'Pairwise Rating Dev': Rating_pairwise_deviation, 
                                          'Pairwise Official Dev': official_pairwise_deviation}, ignore_index=True) 
                  
                  ##### After evaluate the competition, re-calculate participated athletes' Rating
                  # process competition data 
                  competition = competitionFileParser(df, comp_year)
                  # run Rating calculation for this competition  
                  calculateRating(competition.competitors, comp_year)
            
            if show_yearly:
                #print(result)
                print(yearly_result.mean())
                print('\n')
    
    print("----------------------------------------------------------------------")
    # here prints out the summarized results for validation and testing periods respectively
    print('Validation results ({} - {})\n'.format(start_date, validation_end_date))
    print(validation_result.mean())
    #print(validation_result)
    print('\n')
    print('Testing results: \n')
    print(testing_result.mean())
    #print(testing_result)
    print('\n')

In [ ]:
def evaluation(df, official_ranking):
    # get all the athletes who participated in the current competition
    participated_athlete = {}
    participated_official_ranking = pd.DataFrame(columns=['fiscode', 'name', 'points'])

    for fis in df.loc[:,'Fis Code']:
        # get from Rating system
        existed_athlete = athlete_dict.get(fis)
        
        # check if this athlete participated in previous competitions
        # only compare athletes who have competed before
        if existed_athlete:
          participated_athlete[fis] = existed_athlete
          
          # get the athlete data from the offcial ranking data
          official_ranked_athlete = official_ranking[official_ranking['fis code'] == fis]
          # this athlete has not competed in the most recent 52 weeks, which means not on the official ranking. 
          # set Default point to ZERO
          if official_ranked_athlete.empty:
              participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                    'name': existed_athlete.name,
                                                                                    'points':-1}, ignore_index=True)
          else:
              participated_official_ranking = participated_official_ranking.append({'fiscode': fis,
                                                                                    'name': existed_athlete.name,
                                                                                    'points':official_ranked_athlete['points'].values[0]}, ignore_index=True)    
              
    #sort the official ranking results
    participated_official_ranking = participated_official_ranking.sort_values(by=['points'],ascending=False, ignore_index=True)

    # sort the Rating point system
    participated_Rating_ranking = sort_athlete(participated_athlete)

    # filter the athletes who have participated in previous competitions
    df_filtered = df[df['Fis Code'].isin(participated_Rating_ranking['fiscode'])].loc[:,['Athlete Name', 'Fis Code', 'Rank', 'Race Points']]

    Rating_accuracy, official_accuracy = pairwise_accuracy(df_filtered, participated_Rating_ranking, participated_official_ranking)
    Rating_deviation, official_deviation = deviation_accuracy(df_filtered, participated_Rating_ranking, participated_official_ranking)
    Rating_pairwise_deviation, official_pairwise_deviation = pairwise_deviation_accuracy(df_filtered, participated_Rating_ranking, participated_official_ranking)
    
    return Rating_accuracy, official_accuracy, Rating_deviation, official_deviation, Rating_pairwise_deviation, official_pairwise_deviation


In [ ]:
'''
pairwise compare athletes using Elo ranking and the official ranking with the actual competition ranks
This method evaluates whether one athlete ranks higher than others according to the predicted ranks from Elo and Official data, and compare this prediction to the actual competition result
'''
def pairwise_accuracy(competition_ranking_df, Rating_ranking_df, official_ranking_df):    
    count = 0
    Rating_correct = 0
    official_correct = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_Rating = Rating_ranking_df[Rating_ranking_df['fiscode']==athlete1]['rating'].values[0]
        athlete1_official = official_ranking_df[official_ranking_df['fiscode']==athlete1]['points'].values[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_Rating = Rating_ranking_df[Rating_ranking_df['fiscode']==athlete2]['rating'].values[0]
            athlete2_official = official_ranking_df[official_ranking_df['fiscode']==athlete2]['points'].values[0]
            if athlete1_Rating > athlete2_Rating:
                Rating_correct += 1
            if athlete1_official > athlete2_official and athlete2_official != -1:
                official_correct += 1
            count += 1
    
    if count == 0:
        count = 1

    return Rating_correct/count, official_correct/count

In [ ]:
'''
Calculate accuracy by comparing the deviation of the competition rank positions from Elo system and Official rankings to the true competition ranks.
This method evaluates one athlete's predicted rank position his/her true ranks based on Elo and Official data

'''
def deviation_accuracy(competition_ranking_df, Rating_ranking_df, official_ranking_df):    
    count = 0
    Rating_deviation = 0
    official_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0]):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_Rating_rank = Rating_ranking_df[Rating_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
          
        Rating_deviation += abs(i - athlete1_Rating_rank)
        official_deviation += abs(i - athlete1_official_rank)
        count += 1
    return Rating_deviation/count, official_deviation/count

In [ ]:
'''
Calculate accuracy by comparing the pairwise deviation of the rank positions from Elo, Official rankings to the true competition ranks.
This method evaluates the difference between one athlete's predicted rank position to other athletes' and one's true rank to others based on Elo and Official data

'''
def pairwise_deviation_accuracy(competition_ranking_df, Rating_ranking_df, official_ranking_df):    
    count = 0
    Rating_pairwise_deviation = 0
    official_pairwise_deviation = 0
    # Note: some athletes got filtered out b/c they didn't compete before or DNS; 
    # So i used index lookup. Index 1 is 'Fis Code'
    for i in range(competition_ranking_df.shape[0] - 1):
        athlete1 = competition_ranking_df.iloc[i, 1]
        athlete1_Rating_rank = Rating_ranking_df[Rating_ranking_df['fiscode']==athlete1].index[0]
        athlete1_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete1].index[0]
        for j in range(i+1, competition_ranking_df.shape[0]):
            athlete2 = competition_ranking_df.iloc[j, 1]
            athlete2_Rating_rank = Rating_ranking_df[Rating_ranking_df['fiscode']==athlete2].index[0]
            athlete2_official_rank = official_ranking_df[official_ranking_df['fiscode']==athlete2].index[0]
            
            true_diff = i - j
            Rating_pairwise_deviation += abs(true_diff - (athlete1_Rating_rank - athlete2_Rating_rank))
            official_pairwise_deviation += abs(true_diff - (athlete1_official_rank - athlete2_official_rank))
            count += 1
    return Rating_pairwise_deviation/count, official_pairwise_deviation/count

In [ ]:
athlete_dict = {}

# train the Elo point system up until the end_date
train_system(BASE_DIR+'Male', end_date = 20131231)
train_system(BASE_DIR+'Female', end_date = 20131231)

In [ ]:
evaluate_system(BASE_DIR+'Male', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_M, ranking_sheet_names= sheet_names_M, show_yearly=True)

Currently processing file: M_2014.xlsx

Rating accuracy          0.728820
Official accuracy        0.372731
Absolute Rating Dev      4.049952
Absolute Official Dev    4.049249
Pairwise Rating Dev      5.949164
Pairwise Official Dev    6.096599
dtype: float64


Currently processing file: M_2015.xlsx

Rating accuracy          0.721250
Official accuracy        0.540202
Absolute Rating Dev      4.394062
Absolute Official Dev    3.926045
Pairwise Rating Dev      6.478706
Pairwise Official Dev    5.947014
dtype: float64


Currently processing file: M_2016.xlsx

Rating accuracy          0.748338
Official accuracy        0.601795
Absolute Rating Dev      3.790165
Absolute Official Dev    3.648601
Pairwise Rating Dev      5.657049
Pairwise Official Dev    5.494012
dtype: float64


Currently processing file: M_2017.xlsx

Rating accuracy          0.723364
Official accuracy        0.645097
Absolute Rating Dev      5.466491
Absolute Official Dev    5.168251
Pairwise Rating Dev      7.964185
Pairwis

In [ ]:
evaluate_system(BASE_DIR+'Female', start_date = 20140101, validation_end_date = 20171231, xls_ranking = xls_ranking_W, ranking_sheet_names= sheet_names_W, show_yearly=True)

Currently processing file: W_2014.xlsx

Rating accuracy          0.817018
Official accuracy        0.557285
Absolute Rating Dev      1.886425
Absolute Official Dev    1.743739
Pairwise Rating Dev      2.991938
Pairwise Official Dev    2.774364
dtype: float64


Currently processing file: W_2015.xlsx

Rating accuracy          0.627513
Official accuracy        0.491235
Absolute Rating Dev      2.276520
Absolute Official Dev    2.229669
Pairwise Rating Dev      3.534152
Pairwise Official Dev    3.468414
dtype: float64


Currently processing file: W_2016.xlsx

Rating accuracy          0.788359
Official accuracy        0.528335
Absolute Rating Dev      2.517904
Absolute Official Dev    2.668550
Pairwise Rating Dev      3.709294
Pairwise Official Dev    4.069453
dtype: float64


Currently processing file: W_2017.xlsx

Rating accuracy          0.780970
Official accuracy        0.742561
Absolute Rating Dev      3.202901
Absolute Official Dev    2.972451
Pairwise Rating Dev      4.927301
Pairwis